In [43]:
import json
import random

class Player:
    def __init__(self, name, bro, hp, power, accuracy, items, xp=0, level=1):
        self.name = name
        self.bro = bro
        self.hp = hp
        self.max_hp = hp
        self.power = power
        self.accuracy = accuracy
        self.items = items
        self.xp = xp
        self.level = level

class Game:
    FONT_SIZE_SMALL = "\033[1;30m"
    FONT_SIZE_NORMAL = "\033[0m"

    def __init__(self):
        self.items_data = None
        self.characters_data = None
        self.player = None

    def load_items(self):
        try:
            with open("items.json", "r") as file:
                self.items_data = json.load(file)
        except FileNotFoundError:
            print("Error: items.json file not found.")
            self.items_data = []  # Set to an empty list if file not found
        except json.JSONDecodeError:
            print("Error: items.json file is not valid JSON.")
            self.items_data = []  # Set to an empty list if JSON decoding fails

    def load_characters(self):
        try:
            with open("characters.json", "r") as file:
                self.characters_data = json.load(file)
        except FileNotFoundError:
            print("Error: characters.json file not found.")
            self.characters_data = []  # Set to an empty list if file not found
        except json.JSONDecodeError:
            print("Error: characters.json file is not valid JSON.")
            self.characters_data = []  # Set to an empty list if JSON decoding fails

    def save_file(self, data, filename):
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)

    def save_success(self):
        print(f"{self.FONT_SIZE_SMALL}Data saved successfully.")
        print(self.FONT_SIZE_NORMAL)

    def display_slide(self, slide):
        print(slide)
        input("Press Enter to continue...")

    def add_item(self):
        random_item = random.choice(self.items_data)
        print("\nYou have picked up a \033[92m" + random_item["item_name"] + "\n\033[0m")
        return random_item["item_name"]

    def use_item(self, use):
        for item in self.items_data:
            if item["item_name"] == use:
                if item["target"] == "hp":
                    self.player.hp += int(item["effect"])
                    print(use + " was used, your HP is now " + str(self.player.hp))
                elif item["target"] == "power":
                    self.player.power += int(item["effect"])
                    print(use + " was used, your Attack is now " + str(self.player.power))
                elif item["target"] == "accuracy":
                    self.player.accuracy += int(item["effect"])
                    print(use + " was used, your Accuracy is now " + str(self.player.accuracy))
                break
        else:
            print("Item not found.")

    def walk(self):
        encounter = random.randint(1, 8)
        if encounter >= 7:
            self.match()
        elif encounter <= 2:
            item = self.add_item()
            if item in self.player.items:
                self.player.items[item] += 1
            else:
                self.player.items[item] = 1
        elif 2 < encounter < 7:
            print("\nThere is nothing\n")

    def match(self):
        random_enemy = random.choice(self.characters_data)
        print("\nYou have met a level " + str(self.player.level) + " \033[91m" + random_enemy["character_name"] + "\033[0m")

        enemy_hp = random_enemy["hp"] + (random_enemy["hp_up"] * self.player.level)
        enemy_power = random_enemy["power"] + (random_enemy["power_up"] * self.player.level)
        enemy_accuracy = random_enemy["accuracy"]

        while enemy_hp > 0 and self.player.hp > 0:
            print("\nEnemy HP: " + str(enemy_hp) + "\n\nYour HP: " + str(self.player.hp) + "\nYour Attack: " + str(self.player.power) + "\nYour Accuracy: " + str(self.player.accuracy))

            choice = int(input("1: Attack\n2: Use an item\n3: Run\n"))

            if choice == 1:
                chance = random.randint(1, 100)
                if chance <= self.player.accuracy:
                    enemy_hp -= self.player.power
                    print("\033[92m\nYou hit the enemy.")
                else:
                    print("\033[91m\nYou missed the enemy.")

                enemy_chance = random.randint(1, 100)
                if enemy_chance <= enemy_accuracy:
                    self.player.hp -= enemy_power
                    print("\033[91m\nThe enemy hit you.")
                else:
                    print("\033[92m\nThe enemy missed.")
                print("\033[0m")

            elif choice == 2:
                print(self.player.items)
                item_to_use = input("\nWhich item would you like to use?")
                self.use_item(item_to_use)

            elif choice == 3:
                run = random.randint(1, 3)
                if run == 1:
                    print("\033[92m\nYou were able to run away.\n\033[0m")
                    break
                else:
                    print("\033[91m\nYou were not able to run away.\n\033[0m")

        if self.player.hp > 0:
            print("\033[92m\nYou have defeated the enemy.")
            self.player.xp += 35
        else:
            print("\033[91m\nYou have been defeated. Game Over.")
        print("\033[0m")

    def start_game(self):
        self.load_items()
        self.load_characters()

        game = int(input("WELCOME TO POKE-BROS\n\n1: Load a save file \n2: Start a new save file\n"))
        file_name = "save_file.json"

        if game == 1:
            try:
                with open(file_name, 'r') as json_file:
                    data = json.load(json_file)
                    self.player = Player(
                        data["Name"], data["Bro"], data["HP"], data["Power"], data["Accuracy"], data["Items"], data.get("XP", 0), data.get("Level", 1)
                    )
            except FileNotFoundError:
                print("Error: save_file.json file not found.")
                self.player = None
        elif game == 2:
            intro = ["Welcome to the World of Poke-bro!",
                     "In this world you will be capturing and battling against bros all over the world with your own bro.",
                     "Battles can be brutal and fatal so get ready not to shed a tear if you lose your bro."]
            for slide in intro:
                self.display_slide(slide)
            name = input("What is your name? ")
            bro = None
            while bro is None:
                bro_choice = input("Which Bro would you like to accompany you in your journey?\nOffice-Bro\nGym-Bro\nGambler-Bro\n")
                for character in self.characters_data:
                    if character['character_name'] == bro_choice:
                        print(character)
                        confirm = input("Would you like to proceed with this character? (y/n)")
                        if confirm.lower() == "y":
                            broski = character
                            bro = broski["character_name"]
                            hp = broski["hp"]
                            power = broski["power"]
                            accuracy = broski["accuracy"]
                            break
                else:
                    print("Invalid choice. Please select from the available options.")

            items = {"Potion": 1, "Super Potion": 1}
            self.player = Player(name, bro, hp, power, accuracy, items)

            data = {
                "Name": name, "Bro": bro, "HP": hp, "Power": power, "Accuracy": accuracy, "Items": items, "XP": 0, "Level": 1
            }
            self.save_file(data, file_name)
            self.save_success()

    def main(self):
        self.start_game()
        while self.player.hp > 0:
            self.walk()
            if self.player.hp <= 0:
                break
            print(f"Level: {self.player.level}\nXP: {self.player.xp}\nHP: {self.player.hp}\nAttack: {self.player.power}\nAccuracy: {self.player.accuracy}")
            choice = input("1: Walk\n2: Open inventory\n3: Save and exit\n4: Exit\n")
            if int(choice) == 3:
                data = {
                    "Name": self.player.name, "Bro": self.player.bro, "HP": self.player.hp, "Power": self.player.power,
                    "Accuracy": self.player.accuracy, "Items": self.player.items, "XP": self.player.xp, "Level": self.player.level
                }
                self.save_file(data, "save_file.json")
                self.save_success()
                break
            if int(choice) == 4:
                break

if __name__ == "__main__":
    game_instance = Game()
    game_instance.main()


KeyboardInterrupt: Interrupted by user